In [2]:
#Feature Engineering
#creo nuevas variables dentro del mismo mes
#Condimentar a gusto con nuevas variables

#limpio la memoria
rm( list=ls() )
gc()

require("data.table")

#Establezco el Working Directory
setwd("~/")


EnriquecerDataset <- function( dataset , arch_destino )
{
  columnas_originales <-  copy(colnames( dataset ))

  dataset[ , mv_status01       := pmax( Master_status,  Visa_status, na.rm = TRUE) ]
  dataset[ , mv_status02       := Master_status +  Visa_status ]
  dataset[ , mv_status03       := pmax( ifelse( is.na(Master_status), 10, Master_status) , ifelse( is.na(Visa_status), 10, Visa_status) ) ]
  dataset[ , mv_status04       := ifelse( is.na(Master_status), 10, Master_status)  +  ifelse( is.na(Visa_status), 10, Visa_status)  ]
  dataset[ , mv_status05       := ifelse( is.na(Master_status), 10, Master_status)  +  100*ifelse( is.na(Visa_status), 10, Visa_status)  ]

  dataset[ , mv_status06       := ifelse( is.na(Visa_status), 
                                          ifelse( is.na(Master_status), 10, Master_status), 
                                          Visa_status)  ]

  dataset[ , mv_status07       := ifelse( is.na(Master_status), 
                                          ifelse( is.na(Visa_status), 10, Visa_status), 
                                          Master_status)  ]

    
dataset[ , new_weight_cpagodeservicios_ctrx_quarter  := cpagodeservicios*.8+ctrx_quarter*.2  ]
dataset[ , new_payroll_rent_prod_saldo               := paste(ifelse( mpayroll_adj > 0,"1","0"), ifelse( mrentabilidad_adj > 0,"1","0"), ifelse( mcuentas_saldo_adj > 0,"1","0"), ifelse(cproductos > 7,"1","0"))  ]
dataset[ , new_rt_mpasivos_margen_mcuenta_corriente  := ifelse( is.na(mpasivos_margen_adj), 1, mpasivos_margen_adj)  /  ifelse(ifelse( is.na(mcuenta_corriente_adj), 1, mcuenta_corriente_adj)==0,1,mcuenta_corriente_adj) ]
dataset[ , new_limite_saldototal   := (ifelse( is.na(Master_mlimitecompra), 1, Master_mlimitecompra)+ifelse( is.na(Visa_mlimitecompra), 1, Visa_mlimitecompra)) /  ifelse( is.na(Visa_msaldototal_adj), 1, Visa_msaldototal_adj)  ]          
    

    
  columnas_extendidas <-  copy( setdiff(  colnames(dataset), columnas_originales ) )
  

  #grabo con nombre extendido
  fwrite( dataset,
          file=arch_destino,
          sep= "," )
}
#------------------------------------------------------------------------------

#kcarpeta_datasets    <- "../input/laboratorio-de-implementacion-i-2021/"   #KAGGLE
kcarpeta_datasets    <- "./buckets/b1/datasets/"                          #VM o Ubuntu

#Archivo con datos etiquetados para entrenamiento
karchivo_entrada      <-  paste0(kcarpeta_datasets, "competencia2_2022_ADJ.csv.gz")
karchivo_salida      <-  paste0(kcarpeta_datasets, "competencia2_2022_FE.csv.gz")



#lectura rapida del dataset  usando fread  de la libreria  data.table
dataset1  <- fread(karchivo_entrada)

EnriquecerDataset( dataset1, karchivo_salida )



quit( save="no")


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,726367,38.8,1349487,72.1,1349487,72.1
Vcells,183625990,1401.0,781909358,5965.5,812556391,6199.4
